In [1]:
from pyflink.table import (
    EnvironmentSettings,
    TableEnvironment,
    TableDescriptor,
    Schema,
    DataTypes,
    FormatDescriptor,
    EnvironmentSettings
)
from pyflink.table.window import Tumble,Slide, Over
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import expressions as expr
from pyflink.table.expressions import col,call, lit
from pyflink.table.udf import udf

In [2]:
env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)    # Point to the SQL Kafka connector JAR
kafka_connector_jar = "file:///Users/gerardomartinez/DataEng/alpaca/flink-connectors/flink-sql-connector-kafka-3.3.0.jar"

t_env.get_config().set("pipeline.jars", kafka_connector_jar)

dockerC = "localhost:9092"

# t_env.get_config().set("table.exec.emit.early-fire.enabled", "false")
# t_env.get_config().set("table.exec.emit.allow-lateness", "0s")

In [3]:
source_ddl = f"""
        create table Orders(
            proc_time AS PROCTIME(),
            a VARCHAR,
            b BIGINT,
            c BIGINT,
            rowtime TIMESTAMP(3),
            WATERMARK FOR rowtime as rowtime
        ) with (
            'connector' = 'kafka',
            'topic' = 'orders',
            'properties.bootstrap.servers' = '{dockerC}',
            'properties.group.id' = 'transaction_group',
            'scan.startup.mode' = 'earliest-offset',
            'format' = 'json',
            'json.fail-on-missing-field' = 'false',
            'json.ignore-parse-errors' = 'true'
        );
        """

t_env.execute_sql(source_ddl)

sink_ddl = f"""
    create table calaverita (
        window_start TIMESTAMP(3),
        users BIGINT,
        cnt BIGINT
    ) with (
    'connector' = 'kafka',
    'topic' = 'order-streams',
    'properties.bootstrap.servers' = 'localhost:9092',
    'format' = 'json'
    );
    """

t_env.execute_sql(sink_ddl)


# orders\
#     .window(Tumble.over(expr.lit(1).minute).on(col("rowtime")).alias("w")) \
#     .group_by(col("a"), col("w")) \
#     .select(col("a"), col("b").count.alias("cnt")) \
#     .execute_insert("calaverita") \
#     .wait()

# orders\
#     .window(Tumble.over(expr.lit(1).minute).on(col("proc_time")).alias("w")) \
#     .group_by(col("a"), col("w")) \
#     .select(col("a"), col("b").count.alias("cnt")) \
#     .execute()\
#     .print()



In [4]:
# orders\
#     .window(Tumble.over(expr.lit(1).minute).on(col("proc_time")).alias("w")) \
#     .group_by(col("w")) \
#     .select(
#         col("w").start.alias("window_start"),
#         col("a").count.alias("users"),
#         col("b").avg.alias("cnt")
#     ) \
#     .execute()\
#     .print()

query = """
SELECT
    window_start,
    COUNT(a) AS users,
    AVG(b) AS cnt
FROM TABLE(
    TUMBLE(TABLE Orders, DESCRIPTOR(proc_time), INTERVAL '1' MINUTE)
)
GROUP BY window_start, window_end
"""

insert_query = """
INSERT INTO calaverita
SELECT
    window_start,
    COUNT(a) AS users,
    AVG(b) AS cnt
FROM TABLE(
    TUMBLE(TABLE Orders, DESCRIPTOR(proc_time), INTERVAL '1' MINUTE)
)
GROUP BY window_start, window_end
"""


t_results = t_env.execute_sql(insert_query)

In [6]:
t_results.wait()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/gerardomartinez/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [7]:
# # # Define the Kafka source table
# t_env.create_temporary_table(
#     'ordersI',
#     TableDescriptor.for_connector('kafka')
#         .schema(Schema.new_builder()
#                 .column('a', DataTypes.STRING())
#                 .column('b', DataTypes.BIGINT(3))
#                 .column('c', DataTypes.BIGINT(3))
#                 .column('rowtime', DataTypes.TIMESTAMP(3))    
#                 .build())
#         .option('topic', 'orders')
#         .option('properties.bootstrap.servers', dockerC)
#         .option('properties.group.id', 'transaction_group')
#         .option('scan.startup.mode', 'earliest-offset')
#         .format(FormatDescriptor.for_format('json')
#                 .option('fail-on-missing-field', 'false')
#                 .option('ignore-parse-errors', 'true')
#                 .build())
#         .build())

| +I |                     3ngMiBVHdZ |


In [4]:
ordersTable = t_env.from_path("ordersI")

Py4JJavaError: An error occurred while calling o8.from.
: org.apache.flink.table.api.ValidationException: Table `ordersI` was not found.
	at org.apache.flink.table.api.internal.TableEnvironmentImpl.lambda$from$5(TableEnvironmentImpl.java:554)
	at java.base/java.util.Optional.orElseThrow(Optional.java:403)
	at org.apache.flink.table.api.internal.TableEnvironmentImpl.from(TableEnvironmentImpl.java:551)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at org.apache.flink.api.python.shaded.py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at org.apache.flink.api.python.shaded.py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at org.apache.flink.api.python.shaded.py4j.Gateway.invoke(Gateway.java:282)
	at org.apache.flink.api.python.shaded.py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at org.apache.flink.api.python.shad

In [4]:
orders\
    .window(Tumble.over(expr.lit(1).minute).on(col("proc_time")).alias("w")) \
    .group_by(col("w")) \
    .select(
        col("w").start.alias("window_start"),
        col("a").count.alias("users"),
        col("b").avg.alias("cnt")
    ) \
    .execute()\
    .print()

+----+-------------------------+----------------------+----------------------+
| op |            window_start |                users |                  cnt |
+----+-------------------------+----------------------+----------------------+
| +I | 2025-02-07 09:58:00.000 |                   63 |      491394178225294 |
| +I | 2025-02-07 09:59:00.000 |                   12 |      489992853969271 |
| +I | 2025-02-07 10:00:00.000 |                   12 |      637171529236061 |


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/gerardomartinez/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

| +I | 2025-02-07 10:01:00.000 |                    8 |      384036121624407 |


In [4]:
orders.select(col("a")).execute().print()

+----+--------------------------------+
| op |                              a |
+----+--------------------------------+
| +I |                     97QZ0aV5zU |
| +I |                     gkz59P94TY |
| +I |                     X2P3u5mZVM |
| +I |                     SG0U8BusZ8 |
| +I |                     A9egleYpuc |
| +I |                     TibSi5fknR |
| +I |                     3ngMiBVHdZ |
| +I |                     YmTWsWuZ1S |
| +I |                     ppJQm87C5J |


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/gerardomartinez/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

| +I |                     65obVhPiCv |
| +I |                     9AZrM2xWMc |


In [5]:
orders\
    .window(Tumble.over(expr.lit(1).minute).on(col("rowtime")).alias("w")) \
    .group_by(col("a"), col("w")) \
    .select(col("a"), col("b").count.alias("cnt")) \
    .execute_insert("calaverita") \
    .wait()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/gerardomartinez/DataEng/.venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/gerardomartinez/.pyenv/versions/3.10.4/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 